# C# 8 Feature Review

Let's review some of the more [popular features of C# 8](https://docs.microsoft.com/dotnet/csharp/whats-new/csharp-8) that were introduced.

## Readonly members

You can apply the `readonly` modifier to members of a struct. It indicates that the member doesn't modify state. It's more granular than applying the `readonly` modifier to a `struct` declaration. Consider the following mutable struct:

In [ ]:
public struct Point
{
    public double X { get; set; }
    public double Y { get; set; }
    public readonly double Distance => Math.Sqrt(X * X + Y * Y);

    public readonly override string ToString() =>
        $"({X}, {Y}) is {Distance} from the origin";
}

## Default interface methods

You can now add members to interfaces and provide an implementation for those members. This language feature enables API authors to add methods to an interface in later versions without breaking source or binary compatibility with existing implementations of that interface. Existing implementations inherit the default implementation. 

In [ ]:
public class Order
{
		public DateTime Purchased { get; set; }
		public decimal Cost { get; set; }
}

public interface ICustomer
{
	IEnumerable<Order> PreviousOrders { get; }

	DateTime DateJoined { get; }
	DateTime? LastOrder { get; }
	string Name { get; }
	IDictionary<DateTime, string> Reminders { get; }

	public decimal ComputeLoyaltyDiscount()
	{
			DateTime TwoYearsAgo = DateTime.Now.AddYears(-2);
			if ((DateJoined < TwoYearsAgo) && (PreviousOrders.Count() > 5))
			{
					return 0.10m;
			}
			return 0;
	}
}

public interface ICustomer2 : ICustomer {
	decimal ComputeLoyaltyDiscount();
}

public class MyCustomer : ICustomer {

	public IEnumerable<Order> PreviousOrders { get; set; } = new List<Order>();

	public DateTime DateJoined { get; }
	public DateTime? LastOrder { get; }
	public string Name { get; }
	public IDictionary<DateTime, string> Reminders { get; }

}

ICustomer c = new MyCustomer() {
	PreviousOrders = new List<Order> {
		new Order {Cost = 5.99m },
		new Order {Cost = 6.99m },
		new Order {Cost = 7.99m },
		new Order {Cost = 8.99m },
		new Order {Cost = 9.99m },
		new Order {Cost = 11.99m },
	}
};
Console.WriteLine(c.ComputeLoyaltyDiscount());

## Switch expressions

Often, a switch statement produces a value in each of its `case` blocks. Switch expressions enable you to use more concise expression syntax. There are fewer repetitive `case` and `break` keywords, and fewer curly braces. 

In [ ]:
using System.Drawing;

public enum Rainbow
{
    Red,
    Orange,
    Yellow,
    Green,
    Blue,
    Indigo,
    Violet,
		Papayawhip
}

public Color FromRainbow(Rainbow colorBand) =>
    colorBand switch
    {
        Rainbow.Red    => Color.FromArgb(0xFF, 0x00, 0x00),
        Rainbow.Orange => Color.FromArgb(0xFF, 0x7F, 0x00),
        Rainbow.Yellow => Color.FromArgb(0xFF, 0xFF, 0x00),
        Rainbow.Green  => Color.FromArgb(0x00, 0xFF, 0x00),
        Rainbow.Blue   => Color.FromArgb(0x00, 0x00, 0xFF),
        Rainbow.Indigo => Color.FromArgb(0x4B, 0x00, 0x82),
        Rainbow.Violet => Color.FromArgb(0x94, 0x00, 0xD3),
        _              => throw new ArgumentException(message: "invalid enum value", paramName: nameof(colorBand)),
    };

Console.WriteLine(FromRainbow(Rainbow.Red));

## Property patterns

The property pattern enables you to match on properties of the object examined.

In [ ]:
public class Address {
	public string State {get; set;}
	public string City { get; set; }
}

public static decimal ComputeSalesTax(Address location, decimal salePrice) =>
    location switch
    {
        { State: "WA" } => salePrice * 0.06M,
        { State: "MN" } => salePrice * 0.075M,
        { State: "MI" } => salePrice * 0.05M,
				{ State: "PA", City: "Pittsburgh" } => salePrice * 0.07M,
				{ State: "PA", City: "Philadelphia" } => salePrice * 0.08M,
				{ State: "PA" } => salePrice * 0.06M,
        // other cases removed for brevity...
        _ => 0M
    };

var hoagiePrice = 5.99M;
Console.WriteLine(ComputeSalesTax(new Address { State="PA"}, hoagiePrice));

## Tuple patterns

Some algorithms depend on multiple inputs. Tuple patterns allow you to switch based on multiple values expressed as a tuple. The following code shows a switch expression for the game *rock, paper, scissors*:

In [ ]:
public string RockPaperScissors(string first, string second)
    => (first, second) switch
    {
        ("rock", "paper") => "rock is covered by paper. Paper wins.",
        ("rock", "scissors") => "rock breaks scissors. Rock wins.",
				("paper", "rock") => "paper covers rock. Paper wins.",
        ("paper", "scissors") => "paper is cut by scissors. Scissors wins.",
        ("scissors", "rock") => "scissors is broken by rock. Rock wins.",
        ("scissors", "paper") => "scissors cuts paper. Scissors wins.",
				// ("rock", _) => "Good ole rock... Rock always wins",
				// (_, "rock") => "Good ole rock... Rock always wins",
				(_, _) => "tie"
    };

Console.WriteLine(RockPaperScissors("rock", "paper"));
// The Bart Simpson rule:  https://www.youtube.com/embed/b0SoKWLkmLU

## Using declarations

A using declaration is a variable declaration preceded by the using keyword. It tells the compiler that the variable being declared should be disposed at the end of the enclosing scope. 

In [ ]:
static int WriteLinesToFile(IEnumerable<string> lines)
{
    using var file = new System.IO.StreamWriter("WriteLines2.txt");

		int skippedLines = 0;
        foreach (string line in lines)
        {
            if (!line.Contains("Second"))
            {
                file.WriteLine(line);
            }
            else
            {
                skippedLines++;
            }
        }
        return skippedLines;

			}

## Static local functions

You can now add the `static` modifier to local functions to ensure that local function doesn't capture any variables from the enclosing scope.

In [ ]:
int Sum()
{
    int y = 5;
    int x = 7;
    return Add(x, y);

    static int Add(int left, int right) => left + right;
}

Console.WriteLine(Sum());

## Asynchronous disposable

Starting with C# 8.0, the language supports asynchronous disposable types that implement the `System.IAsyncDisposable` interface. You use the `await using` statement to work with an asynchronously disposable object. 

In [ ]:
using System;
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;

public class ExampleAsyncDisposable : IAsyncDisposable, IDisposable
{
    private Utf8JsonWriter _jsonWriter = new(new MemoryStream());

    public void Dispose()
    {
        Dispose(disposing: true);
        GC.SuppressFinalize(this);
    }

    public async ValueTask DisposeAsync()
    {
        await DisposeAsyncCore();

        Dispose(disposing: false);
#pragma warning disable CA1816 // Dispose methods should call SuppressFinalize
        GC.SuppressFinalize(this);
#pragma warning restore CA1816 // Dispose methods should call SuppressFinalize
    }

    protected virtual void Dispose(bool disposing)
    {
        if (disposing)
        {
            _jsonWriter?.Dispose();
        }

        _jsonWriter = null;
    }

    protected virtual async ValueTask DisposeAsyncCore()
    {
        if (_jsonWriter is not null)
        {
            await _jsonWriter.DisposeAsync().ConfigureAwait(false);
        }

        _jsonWriter = null;
    }
}

To properly consume an object that implements the `IAsyncDisposable` interface, you use the `await` and `using` keywords together. Consider the following example, where the `ExampleAsyncDisposable` class is instantiated and then wrapped in an `await using` statement.

In [ ]:
var exampleAsyncDisposable = new ExampleAsyncDisposable();
await using (exampleAsyncDisposable.ConfigureAwait(false))
{
		// Interact with the exampleAsyncDisposable instance.
}

## Indices and ranges

Indices and ranges provide a succinct syntax for accessing single elements or ranges in a sequence.

This language support relies on two new types, and two new operators:

- `System.Index` represents an index into a sequence.
- The index from end operator `^`, which specifies that an index is relative to the end of the sequence.
- `System.Range` represents a sub range of a sequence.
- The range operator `..`, which specifies the start and end of a range as its operands.

Let's start with the rules for indexes. Consider an array `sequence`. The `0` index is the same as `sequence[0]`. The `^0` index is the same as `sequence[sequence.Length]`. Note that `sequence[^0]` does throw an exception, just as `sequence[sequence.Length]` does. For any number `n`, the index `^n` is the same as `sequence.Length - n`.

In [ ]:
var words = new string[]
{
                // index from start    index from end
    "The",      // 0                   ^9
    "quick",    // 1                   ^8
    "brown",    // 2                   ^7
    "fox",      // 3                   ^6
    "jumped",   // 4                   ^5
    "over",     // 5                   ^4
    "the",      // 6                   ^3
    "lazy",     // 7                   ^2
    "dog"       // 8                   ^1
};              // 9 (or words.Length) ^0

In [ ]:
Console.WriteLine($"The last word is {words[^1]}");

In [ ]:
var quickBrownFox = words[1..4];
Console.WriteLine(String.Join(' ', quickBrownFox));

quick brown fox


In [ ]:
var lazyDog = words[^2..^0];
Console.WriteLine(String.Join(' ', lazyDog));

lazy dog


In [ ]:
var allWords = words[..]; // contains "The" through "dog".
var firstPhrase = words[..4]; // contains "The" through "fox"
var lastPhrase = words[6..]; // contains "the", "lazy" and "dog"

Console.WriteLine(String.Join(' ', lastPhrase));

In [ ]:
//Range phrase = 1..4;
var phrase = new Range(1, 4); 
var text = words[phrase];
Console.WriteLine(String.Join(' ', text));

## Null-coalescing assignment

C# 8.0 introduces the null-coalescing assignment operator `??=`. You can use the `??=` operator to assign the value of its right-hand operand to its left-hand operand only if the left-hand operand evaluates to null.

In [ ]:
List<int> numbers = null;
int? i = null;

numbers ??= new List<int>();
numbers.Add(i ??= 17);
numbers.Add(i ??= 20);

Console.WriteLine(string.Join(" ", numbers));  
Console.WriteLine(i);  